In [1]:
import numpy as _np
import matplotlib.pyplot as _plt
import pybdsim as _bd
import h5py as _h5

In [2]:
bdsimfile = "../../../luxe-backgrounds/04_dataLocal/00_LUXEDET1_tracking_for_compton_10000-part.root"
hdf5file = "../../00_ptarmigan/00_luxe_default_100000_particles.h5"

In [3]:
bdsim = _bd.DataPandas.BDSIMOutput(bdsimfile)

In [4]:
last_sampler = bdsim.get_sampler_names()[-1]

In [5]:
df = bdsim.get_sampler(last_sampler)

In [6]:
df

,file_idx,event_idx,x,xp,y,yp,z,zp,T,energy,partID,trackID,weight
0,0,0,4.823656e-06,1.581510e-06,6.507235e-06,-0.000001,0.0,1.0,25.040825,13.999986,11,1,1.0
1,0,1,-6.857778e-06,-2.930807e-06,-1.307786e-05,0.000005,0.0,1.0,25.040825,14.000026,11,1,1.0
2,0,2,1.580387e-05,-6.708624e-07,-1.032120e-06,0.000003,0.0,1.0,25.040825,13.999988,11,1,1.0
3,0,3,-9.703575e-06,-5.154786e-06,8.806214e-06,-0.000006,0.0,1.0,25.040825,13.999999,11,1,1.0
4,0,4,5.804243e-07,-3.401074e-06,-4.123114e-06,0.000005,0.0,1.0,25.040825,13.999996,11,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,9995,2.864020e-06,7.692836e-06,1.442006e-05,-0.000002,0.0,1.0,25.040825,13.999996,11,1,1.0
9996,0,9996,1.993116e-05,-2.365524e-09,6.772235e-06,0.000001,0.0,1.0,25.040825,13.999980,11,1,1.0
9997,0,9997,-3.770349e-06,-1.311004e-06,1.497604e-07,0.000005,0.0,1.0,25.040825,14.000008,11,1,1.0
9998,0,9998,1.680506e-05,-4.766686e-06,-6.869508e-06,-0.000002,0.0,1.0,25.040825,13.999990,11,1,1.0


In [85]:
particle_dict={11:  {"Name": "electron", "mass": 0.511e-3},
               -11: {"Name": "positron", "mass": 0.511e-3},
               22:  {"Name": "photon",   "mass": 0}}

def getBdsimFinalData(bdsimfile, mu_z=0, sigma_z=25e-12):
    bdsim = _bd.DataPandas.BDSIMOutput(bdsimfile)
    last_sampler = bdsim.get_sampler_names()[-1]
    df = bdsim.get_sampler(last_sampler)
    getMassInDF(df)
    for partID in particle_dict.keys():
        df.loc[df['partID'] == partID, 'mass'] = particle_dict[partID]['mass']

    df['Px'] = df['xp']*_np.sqrt(pow(df['energy'], 2) - pow(df['mass'], 2))
    df['Py'] = df['yp']*_np.sqrt(pow(df['energy'], 2) - pow(df['mass'], 2))
    df['Pz'] = _np.sqrt(pow(df['energy'], 2) - pow(df['Px'], 2) - pow(df['Py'], 2) - pow(df['mass'], 2))

    df['z'] = _np.array([_np.random.normal(mu_z, sigma_z) for i in range(len(df['x']))])
    
    # We assume that the spacetime interval s^2 is 0
    df['T'] = _np.sqrt(pow(df['x'], 2) + pow(df['y'], 2) + pow(df['z'], 2))
    
    return df
    
def getBdsimDataInH5(bdsimfile, hdf5file, mu_z=0, sigma_z=25e-12):
    df = getBdsimFinalData(bdsimfile, mu_z, sigma_z)
    
    h5 = _h5.File(hdf5file, 'w')
    h5["beam_axis"] = "+z"
    h5['config/unit/momentum'] = "GeV/c"
    h5['config/unit/position'] = "m"
    
    for partID in df['partID'].unique():
        npart = len(df[df['partID'] == partID])
        partname = particle_dict[partID]['Name']
        
        Momentum = _np.array([])
        Position = _np.array([])
        Weight = _np.array([])
        for i in df.index:
            mom4vect = _np.array([df.iloc[i]['energy'], df.iloc[i]['Px'], df.iloc[i]['Py'], df.iloc[i]['Pz']])
            pos4vect = _np.array([df.iloc[i]['T'],      df.iloc[i]['x'],  df.iloc[i]['y'],  df.iloc[i]['z']])
            
            Momentum = _np.append(Momentum, mom4vect)
            Position = _np.append(Position, pos4vect)
            Weight = _np.append(Weight, df.iloc[i]['weight'])
        
        h5.create_dataset('final-state/{}/momentum'.format(partname), data=_np.reshape(Momentum, (npart, 4)))
        h5.create_dataset('final-state/{}/position'.format(partname), data=_np.reshape(Position, (npart, 4)))
        h5.create_dataset('final-state/{}/weight'.format(partname), data=Weight)
            
    h5.close()

In [86]:
getBdsimDataInH5(bdsimfile, "testofthefunction.h5")

In [87]:
test = _h5.File("testofthefunction.h5")

In [88]:
test["final-state/electron/position"][()]

array([[ 8.10010933e-06,  4.82365613e-06,  6.50723541e-06,
         2.98818388e-11],
       [ 1.47668391e-05, -6.85777786e-06, -1.30778599e-05,
         2.16735720e-11],
       [ 1.58375326e-05,  1.58038656e-05, -1.03211971e-06,
         5.12431663e-11],
       ...,
       [ 3.77332251e-06, -3.77034939e-06,  1.49760425e-07,
         1.93981631e-11],
       [ 1.81548951e-05,  1.68050610e-05, -6.86950807e-06,
         3.94551514e-12],
       [ 3.23680235e-06, -2.44971375e-06,  2.11560678e-06,
        -1.67305460e-12]])

In [89]:
test.close()

In [96]:
def getMass(particle):
    if particle == 'electron' or particle == 'positron':
        return 0.511e-3
    elif particle == 'photon':
        return 0
    else:
        raise ValueError("Unknown particle '{}'".format(particle))

def getPtarmiganDataInDict(hdf5file):
    h5 = _h5.File(hdf5file)
    data_dict = {}
    for particle in h5['final-state'].keys():
        if particle != 'laser':
            data_dict[particle] = {}
            data_dict[particle]['momentum'] = h5['final-state/{}/momentum'.format(particle)][()]
            data_dict[particle]['position'] = h5['final-state/{}/position'.format(particle)][()]
            data_dict[particle]['weight'] = h5['final-state/{}/weight'.format(particle)][()]
    return data_dict
    h5.close()
    
def getPtarmiganDataInBDSIM(hdf5file, outputfilename='beam.dat', particle='electron'):
    data_dict = getPtarmiganDataInDict(hdf5file)
    for particle in data_dict.keys():
        weight = data_dict[particle]['weight'][:]
    
        energy = data_dict[particle]['momentum'][:][:, 0]
        momX   = data_dict[particle]['momentum'][:][:, 1]
        momY   = data_dict[particle]['momentum'][:][:, 2]
    
        posX   = data_dict[particle]['position'][:][:, 1]
        posY   = data_dict[particle]['position'][:][:, 2]
        posZ   = data_dict[particle]['position'][:][:, 3]
    
        angX   = momX / _np.sqrt(pow(energy, 2) - pow(getMass(particle), 2))
        angY   = momY / _np.sqrt(pow(energy, 2) - pow(getMass(particle), 2))
    
        file = open(outputfilename, 'w')
        for i in range(len(energy)):
            file.write("{} {} {} {} {} {} \n".format(posX[i], angX[i], posY[i], angY[i], posZ[i], energy[i], weight))
    file.close()

In [69]:
getPtarmiganDataInBDSIM(hdf5file, particle='photon')

In [70]:
h5 = _h5.File(hdf5file)

In [71]:
h5['final-state/electron/momentum'][()]

array([[ 8.03240788e+00, -1.03929464e-04, -6.66420187e-04,
         8.03240784e+00],
       [ 1.62578210e+01,  2.31566844e-04,  1.81234661e-04,
         1.62578210e+01],
       [ 1.59601466e+01, -1.87268790e-05, -1.36179725e-04,
         1.59601466e+01],
       ...,
       [ 6.57956515e+00,  4.82632344e-04, -3.18834263e-04,
         6.57956511e+00],
       [ 1.10918227e+01,  3.91197399e-04, -1.30487752e-04,
         1.10918227e+01],
       [ 9.66187620e+00, -3.41587530e-04,  2.75481195e-04,
         9.66187617e+00]])

In [77]:
ID = h5['final-state/electron/id'][()]

In [78]:
parentID = h5['final-state/electron/parent_id'][()]

In [81]:
_np.unique(ID == parentID)

array([False,  True])

In [111]:
ID

array([     2,      7,     11, ..., 162392, 162394, 162399], dtype=uint64)